<h1>Part 1</h1>

First let us import all the packages and libraries we will be using in this project.

In [153]:
import pandas as pd
import requests

In [154]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [155]:
import lxml

Now, we will load the data from the given Wikipedia page by using get method and then use read_html which stores all the tables from html page to a list of dataframes.

In [156]:
toronto_data=pd.read_html(requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").content)

As we got our list of dataframes, we will select the dataframe required for our project. Here we require the dataframe stored at index 0.  

In [157]:
toronto_data=toronto_data[0]

In [158]:
toronto_data

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Now, we will drop all the rows where Borough==Not assigned and then reseting our index values.

In [159]:
df=toronto_data['Borough']!='Not assigned'
df

0      False
1      False
2       True
3       True
4       True
       ...  
175    False
176    False
177    False
178     True
179    False
Name: Borough, Length: 180, dtype: bool

In [160]:
toronto_data=toronto_data[df]
toronto_data=toronto_data.reset_index(drop=True)
toronto_data.head(20)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"



Now, let's check the shape of our dataframe.

In [161]:
toronto_data.shape

(103, 3)

<h1>Part 2</h1>

In this part we have to extract the coordinates for each neighbourhood. For this we have two methods: 1.through python geocoder package and 2.Get the already stored coordinates from csv file.
You can use the code given below to extract the latitude and longitude of neighbourhoods using geocoder. But here we will use the csv file to get our coordinates and will try to store the coordinates into our dataframe i.e. <b>toronto_data</b> so that we can use these coordinates to access Foursquare API.

<p>!conda install -c conda-forge geocoder --yes 
   
import geocoder
<pre>
for postal,borough,neigh in zip(toronto_data['Postal Code'],toronto_data['Borough'],toronto_data['Neighbourhood']):
    lat_lng_coords = None<br>
    while(lat_lng_coords is None):
            g = geocoder.google('M3A,North York,Parkwoods')
            lat_lng_coords = g.latlng
            
    toronto_data['latitude'] = lat_lng_coords[0]
    toronto_data['longitude'] = lat_lng_coords[1]
    </pre></p>
    

In [162]:
file=pd.read_csv("Geospatial_Coordinates.csv")

In [163]:
file.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [164]:
toronto_data=pd.merge(toronto_data,file,how='inner',on='Postal Code')

In [165]:
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1>Part 3</h1>

In this section we will be exploring neighbourhoods in Toronto using the Foursquare API.
Before that let us visualize neighbourhoods on map.

In [166]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim  

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [167]:
import folium

In [168]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [169]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now let's explore neighbourhoods in Toronto. For this we will need our Foursquare credentials.

In [170]:
CLIENT_ID = 'KT041LGZIHDSIQAHBY5MCJW0SADKWYEBVSVXKT3QD1YX43RX' # your Foursquare ID
CLIENT_SECRET = 'QXE4WCDW13XVOPAWY5UV3DKWMZGVOTZA0JKX5S5KJWJ1M1N3' # your Foursquare Secret
VERSION = '20180605'
LIMIT=100

Define a function that uses these credentials and returns all the venues(Max=100) around each neighborhood in Toronto.

In [171]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        

In [172]:
venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [173]:
#check how many venues for each neighbourhood
venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,3,3,3,3,3,3


In [174]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 269 uniques categories.


In [175]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot.drop('Neighborhood',axis=1,inplace=True)

Here in above dataframe we created we don,t know the actual index of Neighborhood so let us drop it first and then we will add it again to first column.

In [176]:
# add neighborhood column back to dataframe
onehot.insert(0,column='Neighbourhood',value=venues['Neighbourhood'])
onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [177]:
onehot.shape

(2124, 269)

In [178]:
grouped = onehot.groupby('Neighbourhood').mean().reset_index()
#most_common=grouped
grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


Let's check the 10 most common venues in each neighbourhood.

In [179]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [215]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = grouped['Neighbourhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Mobile Phone Shop,Pizza Place,Supermarket,Deli / Bodega,Sushi Restaurant,Ice Cream Shop,Shopping Mall,Restaurant
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Restaurant,Sandwich Place,Coffee Shop,Indian Restaurant,Pub,Butcher,Sushi Restaurant,Breakfast Spot,Cupcake Shop


Now let us performing clustering of neighborhoods using KMeans Clustering technique

In [216]:
from sklearn.cluster import KMeans


In [217]:
kclusters = 5

grouped_clustering = grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [218]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Portuguese Restaurant,Pizza Place,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2.0,Coffee Shop,Park,Café,Pub,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Beer Store,Shoe Store
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,2.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Dog Run,Dim Sum Restaurant,Diner
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2.0,Coffee Shop,Yoga Studio,Music Venue,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant,College Auditorium


In [225]:
#merged['Cluster Labels'].fillna(0)
merged['Cluster Labels'].dropna(inplace=True) 

Now we will plot the clusters on the map.

In [226]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [227]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighbourhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now let us examine each cluster!
Here you can see that neighbourhoods belonging to same borough 

In [219]:
merged.loc[merged['Cluster Labels']== 0, merged.columns[[1,2] + list(range(5, merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,Victoria Village,0.0,Hockey Arena,Portuguese Restaurant,Pizza Place,Coffee Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
8,East York,"Parkview Hill, Woodbine Gardens",0.0,Pizza Place,Gym / Fitness Center,Café,Athletics & Sports,Intersection,Gastropub,Bank,Pharmacy,Breakfast Spot,Dumpling Restaurant
10,North York,Glencairn,0.0,Park,Pizza Place,Sushi Restaurant,Pub,Japanese Restaurant,Yoga Studio,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
50,North York,Humber Summit,0.0,Furniture / Home Store,Pizza Place,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
56,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.0,Turkish Restaurant,Coffee Shop,Sandwich Place,Discount Store,Doner Restaurant,Dim Sum Restaurant,Diner,Distribution Center,Dog Run,Yoga Studio
63,York,"Runnymede, The Junction North",0.0,Grocery Store,Pizza Place,Bus Line,Brewery,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
67,Central Toronto,Davisville North,0.0,Park,Gym,Pizza Place,Breakfast Spot,Hotel,Sandwich Place,Department Store,Food & Drink Shop,Discount Store,Dim Sum Restaurant
70,Etobicoke,Westmount,0.0,Pizza Place,Discount Store,Middle Eastern Restaurant,Intersection,Coffee Shop,Sandwich Place,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Diner
72,North York,"Willowdale, Willowdale West",0.0,Grocery Store,Bank,Pharmacy,Pizza Place,Coffee Shop,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store
77,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",0.0,Park,Sandwich Place,Bus Line,Mobile Phone Shop,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,College Rec Center,Distribution Center


In [221]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1,2] + list(range(5, merged.shape[1]))]]


,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,"Humberlea, Emery",1.0,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Fast Food Restaurant
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",1.0,Baseball Field,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore,Fast Food Restaurant


In [222]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1,2] + list(range(5, merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"Regent Park, Harbourfront",2.0,Coffee Shop,Park,Café,Pub,Bakery,Theater,Breakfast Spot,Performing Arts Venue,Beer Store,Shoe Store
3,North York,"Lawrence Manor, Lawrence Heights",2.0,Furniture / Home Store,Clothing Store,Accessories Store,Boutique,Event Space,Coffee Shop,Vietnamese Restaurant,Dog Run,Dim Sum Restaurant,Diner
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",2.0,Coffee Shop,Yoga Studio,Music Venue,Beer Bar,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Persian Restaurant,College Auditorium
6,Scarborough,"Malvern, Rouge",2.0,Fast Food Restaurant,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
7,North York,Don Mills,2.0,Gym,Beer Store,Japanese Restaurant,Coffee Shop,Restaurant,Grocery Store,Chinese Restaurant,Sandwich Place,Bike Shop,Sporting Goods Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,"First Canadian Place, Underground city",2.0,Coffee Shop,Café,Hotel,Gym,Japanese Restaurant,Restaurant,Asian Restaurant,American Restaurant,Seafood Restaurant,Steakhouse
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",2.0,River,Smoke Shop,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
99,Downtown Toronto,Church and Wellesley,2.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Café,Men's Store,Mediterranean Restaurant,Yoga Studio
100,East Toronto,"Business reply mail Processing Centre, South C...",2.0,Light Rail Station,Garden Center,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market,Fast Food Restaurant


In [223]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1,2] + list(range(5, merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,3.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
16,York,Humewood-Cedarvale,3.0,Hockey Arena,Trail,Park,Field,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
21,York,Caledonia-Fairbanks,3.0,Park,Women's Store,Pool,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
32,Scarborough,Scarborough Village,3.0,Playground,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
52,North York,"Willowdale, Newtonbrook",3.0,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
66,North York,York Mills West,3.0,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
85,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",3.0,Park,Playground,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
91,Downtown Toronto,Rosedale,3.0,Park,Playground,Trail,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [224]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1,2] + list(range(5, merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,"Moore Park, Summerhill East",4.0,Tennis Court,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio,Department Store
